In [1]:
from data.process import *

2024-07-17 15:15:38.742957: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-17 15:15:38.751370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 15:15:38.763136: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 15:15:38.763168: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 15:15:38.770902: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_dataframe()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
import xgboost as xgb
from model.xgb_opt import *

In [4]:
xgboost_params = {
    **xgb_fixed_params,
    # fixed after secone optimization
    'colsample_bytree': 0.21039136022674532,
    'max_depth': 27,
    'min_child_weight': 36,
    # override fixed params
    'n_estimators': 5000,
    'early_stopping_rounds': 200,
}

In [5]:
model = xgb.XGBClassifier(
    **xgboost_params,
)

boosting = model.fit(X_train, Y_train, eval_set=[(X_dev, Y_dev)], verbose=50)

[0]	validation_0-auc:0.76156
[50]	validation_0-auc:0.87807
[100]	validation_0-auc:0.88163
[150]	validation_0-auc:0.88553
[200]	validation_0-auc:0.88730
[250]	validation_0-auc:0.88835
[300]	validation_0-auc:0.88860
[350]	validation_0-auc:0.88876
[400]	validation_0-auc:0.88879
[450]	validation_0-auc:0.88914
[500]	validation_0-auc:0.88933
[550]	validation_0-auc:0.88943
[600]	validation_0-auc:0.88946
[650]	validation_0-auc:0.88945
[700]	validation_0-auc:0.88949
[750]	validation_0-auc:0.88963
[800]	validation_0-auc:0.88973
[850]	validation_0-auc:0.88971
[900]	validation_0-auc:0.88983
[950]	validation_0-auc:0.88991
[1000]	validation_0-auc:0.88986
[1050]	validation_0-auc:0.88976
[1100]	validation_0-auc:0.88969
[1150]	validation_0-auc:0.88955
[1183]	validation_0-auc:0.88948


In [6]:
model.get_booster().save_model("xgboost_model_new.json")